In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import string
import numpy as np
import json

from collecitons import Counter
from pathlib import Path
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [ ]:
from yelp.vocab import Vocabulary
from yelp.vectorizer import Vectorizer
from yelp.dataset import ProjectDataset

In [ ]:
path = Path('./data/yelp')
review_csv = path/'reviews_with_splits_lite.csv'
scratch = path/'scratch'
vectorizer_path = scratch/'vectorizer.json'

In [ ]:
review_vocab = Vocabulary(add_unk=True)

In [ ]:
review_df = pd.read_csv(review_csv)
train_df = review_df[review_df['split'] == 'train']
vectorizer = Vectorizer.from_dataframe(train_df, cutoff=25)
vocab = vectorizer.review_vocab

In [ ]:
dataset = ProjectDataset.load_data_and_create_vectorizer(review_csv)
dataset.save_vectorizer(vectorizer_path)
# dataset = ProjectDataset.load_data_and_vectorizer(review_csv, vectorizer_path)

In [ ]:
dataset.set_split('train')
dataloader = DataLoader(dataset, batch_size=1)

In [ ]:
itr = iter(dataloader)

In [ ]:
out = next(itr)

In [ ]:
x, y = out['x_data'], out['y_target']

In [ ]:
x,y

In [ ]:
nx = x.numpy()

In [ ]:
np.where(nx==1)

In [ ]:
dataset._vectorizer.review_vocab.lookup_idx(72)

In [ ]:
df.iloc[0]['review']

In [ ]:
dataset._lookup_dict.keys['train'][0].

In [ ]:
next(itr)

In [ ]:
def getit():
    for d in dataloader:
        out = {}
        for name, tensor in d.items():
            out[name] = d[name]
        yield out

In [ ]:
itr = iter(getit())

In [ ]:
next(itr)

In [ ]:
!ls {path}

In [ ]:
df = pd.read_csv(path/'reviews_with_splits_lite.csv')

In [ ]:
df.head()

In [ ]:
df.iloc[16]